In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import pickle
import numpy as np
from keras.models import load_model
import json
import random


Using TensorFlow backend.
/Users/chuckclarke/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/chuckclarke/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/chuckclarke/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/chuckclarke/opt/ana

In [2]:
from PyQt5 import QtWidgets, uic,QtCore
import sys

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

In [4]:
class Ui(QtWidgets.QMainWindow):
    
    intents=None
    model= None
    words=[]
    classes = []
    documents = []
    train_x= None 
    train_y = None
    
    def __init__(self, parent = None):
        super(Ui, self).__init__() # Call the inherited classes __init__ method
        uic.loadUi('chatbot_user.ui', self) # Load the .ui file
        
        
        
        self.button = self.findChild(QtWidgets.QPushButton, 'pushButton_send')
        self.button.setStyleSheet("background-color: lawngreen")
        self.button.clicked.connect(self.send)
        
        self.button_import = self.findChild(QtWidgets.QPushButton, 'pushButton__import_json')
        self.button_import.setStyleSheet("background-color: lightgreen")
        self.button_import.clicked.connect(self.open_file)
        
        self.button_model = self.findChild(QtWidgets.QPushButton, 'pushButton__run_save_model')
        self.button_model.setStyleSheet("background-color: red")
        self.button_model.clicked.connect(self.create_training_data)
        
        self.button_load_save_model = self.findChild(QtWidgets.QPushButton, 'pushButton_load_save_model')
        self.button_load_save_model.setStyleSheet("background-color: blue")
        self.button_load_save_model.clicked.connect(self.load_save_objects)
        
        self.lineEdit_chatbot =self.findChild(QtWidgets.QLineEdit,'lineEdit_chatbot')
        self.lineEdit_user =self.findChild(QtWidgets.QLineEdit,'lineEdit__user')
        
        
        self.tableWidget  =self.findChild(QtWidgets.QTableWidget,'tableWidget_chatbot')
        
        
       
        self.tableWidget.setHorizontalHeaderLabels(["User Response","Chatbot Response","Accurancy"])
        
        
        
        
        
        self.msg = QtWidgets.QMessageBox()
        self.msg.setWindowTitle("Info")
        self.msg.setText("End of Model Run")
        
        self.show() # Show the GUI
        
    def open_file(self):
        intents_file = open('intents.json').read()
        self.intents = json.loads(intents_file)
        self.process_data(self.intents)  
        
        
    def process_data(self,intents):
    
       
        for intent in self.intents['intents']:
             for pattern in intent['patterns']:
                word = nltk.word_tokenize(pattern)
                self.words.extend(word)        
                #add documents in the corpus
                self.documents.append((word, intent['tag']))
                # add to our classes list
                if intent['tag'] not in self.classes:
                    self.classes.append(intent['tag'])
        print(self.documents)  
        self.Lemmatization()
        
    def load_save_objects(self):
        
        self.intents = json.loads(open('intents.json').read())
        self.words = pickle.load(open('words.pkl','rb'))
        self.model = load_model('chatbot_model.h5')
        self.classes = pickle.load(open('classes.pkl','rb'))  
        
    def Lemmatization(self):
        ignore_letters = ['!', '?', ',', '.']  
        # lemmaztize and lower each word and remove duplicates
        self.words = [lemmatizer.lemmatize(w.lower()) for w in self.words if w not in ignore_letters]
        self.words = sorted(list(set(self.words)))
        self.classes = sorted(list(set(self.classes)))
        # documents = combination between patterns and intents
        print (len(self.documents), "documents")
        # classes = intents
        print (len(self.classes), "classes", self.classes) 
        # words = all words, vocabulary
        print (len(self.words), "unique lemmatized words", self.words)
        pickle.dump(self.words,open('words.pkl','wb'))
        pickle.dump(self.classes,open('classes.pkl','wb'))
        
    def create_model(self):
        
        # deep neural networds model
             
            self.msg.setWindowTitle("Info")
            self.msg.setText("Start of Model Run. This may take about 3 to 10 minutes.")
            x = self.msg.exec_()  # this will show our messagebox
            self.model = Sequential()
            self.model.add(Dense(128, input_shape=(len(self.train_x[0]),), activation='relu'))
            self.model.add(Dropout(0.5))
            self.model.add(Dense(64, activation='relu'))
            self.model.add(Dropout(0.5))
            self.model.add(Dense(len(self.train_y[0]), activation='softmax'))
        # Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
            sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
            self.model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

         #Training and saving the model 
            hist = self.model.fit(np.array(self.train_x), np.array(self.train_y), epochs=200, batch_size=5, verbose=1)
            self.model.save('chatbot_model.h5', hist)
            print("model is created")
            self.msg.setWindowTitle("Info")
            self.msg.setText("End of Model Run")
            x = self.msg.exec_()  # this will show our messagebox
    def clean_up_sentence(self,sentence):
              # tokenize the pattern - splitting words into array
                sentence_words = nltk.word_tokenize(sentence)
             # stemming every word - reducing to base form
                sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
                return sentence_words  
            
    def bag_of_words(self,sentence, words, show_details=True):

           # tokenizing patterns
             sentence_words = self.clean_up_sentence(sentence)
          # bag of words - vocabulary matrix
             bag = [0]*len(words)  
             for s in sentence_words:
                for i,word in enumerate(words):
                     if word == s: 
                    # assign 1 if current word is in the vocabulary position
                         bag[i] = 1
                         if show_details:
                              print ("found in bag: %s" % word)
             return(np.array(bag))   

    def predict_class(self,sentence):
              # filter below  threshold predictions
              p = self.bag_of_words(sentence, self.words,show_details=False)
              res = self.model.predict(np.array([p]))[0]
              ERROR_THRESHOLD = 0.25
              results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
              # sorting strength probability
              results.sort(key=lambda x: x[1], reverse=True)
              return_list = []
              for r in results:
                  return_list.append({"intent": self.classes[r[0]], "probability": str(r[1])})
              return  return_list   
    
    def update_datagrid(self,user,chatbot, prec):
        
        
          self.numRows = self.tableWidget.rowCount()
          self.tableWidget.insertRow(self.numRows) 
          self.tableWidget.setItem(self.numRows, 0, QtWidgets.QTableWidgetItem(user))
          self.tableWidget.setItem(self.numRows, 1, QtWidgets.QTableWidgetItem(chatbot))
          self.tableWidget.setItem(self.numRows, 2, QtWidgets.QTableWidgetItem(prec))
        
    def send(self):
        if self.model == None :
            self.msg.setWindowTitle("Info")
            self.msg.setText("Please Run Model or Load Save Model")
            x = self.msg.exec_()  # this will show our messagebox
            self.lineEdit_chatbot.setText("  ")
            return 
        else :
            msg =self.lineEdit_user.text()
        
        if msg != '':
            ints = self.predict_class(msg)
            res = self.getResponse(ints, self.intents)
           
            self.lineEdit_user.setText("")
            self.lineEdit_chatbot.setFocus()
            self.lineEdit_chatbot.setText(res)
            self.lineEdit_chatbot.setFocus()
            self.lineEdit_user.setFocus()
            self.update_datagrid(msg,res," ")
           
            
            print(res)      
     
    def getResponse(self,ints, intents_json):
            tag = ints[0]['intent']
            list_of_intents = intents_json['intents']
            for i in list_of_intents:
                if(i['tag']== tag):
                    result = random.choice(i['responses'])
                    break
            return result
        
    def create_training_data(self):
        # create the training data
        training = []
       # create empty array for the output
        output_empty = [0] * len(self.classes)
       # training set, bag of words for every sentence
        for doc in self.documents:
            bag = []
            word_patterns = doc[0]
            word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
            for word in self.words:
                bag.append(1) if word in word_patterns else bag.append(0)
                # output is a '0' for each tag and '1' for current tag (for each pattern)
                output_row = list(output_empty)
                output_row[self.classes.index(doc[1])] = 1
                training.append([bag, output_row])
               # shuffle the features and make numpy array
        random.shuffle(training)
        training = np.array(training)
              # create training and testing lists. X - patterns, Y - intents
        self.train_x = list(training[:,0])
        self.train_y = list(training[:,1])
        print("Training data is created")
        self.create_model()
    
    

In [ ]:
app = QtWidgets.QApplication(sys.argv)
window = Ui()
app.exec_()


[(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Whats', 'up'], 'greeting'), (['cya'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['how', 'old'], 'age'), (['how', 'old', 'is', 'tim'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['how', 'old', 'are', 'you'], 'age'), (['age', '?'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you'], 'name'), (['whats', 'your', 'name', '?'], 'name'), (['Id', 'like', 'to', 'buy', 'something'], 'shop'), (['whats', 'on', 'the', 'menu'], 'shop'), (['what', 'do', 'you', 'reccommend', '?'], 'shop'), (['could', 'i', 'get', 'something', 'to', 'eat'], 'shop'), (['when', 'are', 'you', 'guys', 'open'], 'hours'), (['what', 'are', 'your', 'hours'], 'hours'), (['hours', 'of', 'operation'], 'hours')]
